# 10 SAXS Presets

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/imewei/NLSQ/blob/main/examples/notebooks/08_workflow_system/10_saxs_presets.ipynb)

SAXS (Small-Angle X-ray Scattering) Domain Preset Example.

This example demonstrates how to create a custom preset for SAXS form factor
fitting using NLSQ's kwargs factory pattern.

SAXS analysis typically involves:
- Fitting form factor models (spheres, cylinders, core-shell, etc.)
- Parameters spanning many orders of magnitude (radius in nm, intensity in counts)
- High precision requirements for structural characterization
- Moderate dataset sizes (typically hundreds to thousands of q-points)

Run this example:
    python examples/scripts/08_workflow_system/10_saxs_presets.py

In [ ]:
# @title Install NLSQ (run once in Colab)
import sys

if 'google.colab' in sys.modules:
    print("Running in Google Colab - installing NLSQ...")
    !pip install -q nlsq
    print("NLSQ installed successfully!")
else:
    print("Not running in Colab - assuming NLSQ is already installed")

In [ ]:
import jax.numpy as jnp
import numpy as np

from nlsq import fit

In [ ]:
def create_saxs_preset() -> dict:
    """Create a workflow configuration optimized for SAXS form factor fitting.

    SAXS-specific considerations:
    - Tolerances: 1e-9 for accurate structural parameters
    - Multi-start: Helpful for complex form factors with shape ambiguity
    - Normalization: Critical due to intensity/size parameter scale differences

    Returns
    -------
    dict
        kwargs for fit() optimized for SAXS analysis.

    Example
    -------
    >>> kwargs = create_saxs_preset()
    >>> kwargs['gtol']
    1e-09
    >>> kwargs['multistart']
    True
    """
    return {
        "workflow": "standard",
        "gtol": 1e-9,
        "ftol": 1e-9,
        "xtol": 1e-9,
        "multistart": True,
        "n_starts": 12,
        "sampler": "lhs",
    }


def sphere_form_factor(q, radius, scale, background):
    """Sphere form factor for SAXS fitting.

    The form factor amplitude for a uniform sphere:
        F(q) = 3 * [sin(qR) - qR*cos(qR)] / (qR)^3

    Intensity: I(q) = scale * |F(q)|^2 + background
    """
    qr = q * radius

    form_factor = jnp.where(
        jnp.abs(qr) < 1e-6,
        1.0 - qr**2 / 10.0,
        3.0 * (jnp.sin(qr) - qr * jnp.cos(qr)) / qr**3,
    )

    return scale * form_factor**2 + background


def core_shell_form_factor(q, r_core, r_shell, scale, background):
    """Core-shell sphere form factor for SAXS fitting."""

    def sphere_amplitude(q_val, r):
        qr = q_val * r
        return jnp.where(
            jnp.abs(qr) < 1e-6,
            1.0 - qr**2 / 10.0,
            3.0 * (jnp.sin(qr) - qr * jnp.cos(qr)) / qr**3,
        )

    v_core = (4.0 / 3.0) * jnp.pi * r_core**3
    v_shell = (4.0 / 3.0) * jnp.pi * r_shell**3

    f_core = v_core * sphere_amplitude(q, r_core)
    f_shell = v_shell * sphere_amplitude(q, r_shell)

    f_total = f_core + 0.5 * (f_shell - f_core)

    return scale * (f_total / v_shell) ** 2 + background

In [ ]:
def main():
    print("=" * 70)
    print("SAXS Domain Preset Example")
    print("=" * 70)
    print()

    # Create the SAXS preset
    kwargs = create_saxs_preset()

    print("SAXS Preset Configuration:")
    print("-" * 40)
    for key, value in kwargs.items():
        print(f"  {key}: {value}")
    print()

    # Generate synthetic SAXS data for sphere form factor
    print("Generating synthetic SAXS data (spheres)...")
    np.random.seed(42)

    q_data = np.logspace(-2, 0, 200)

    true_params = {
        "radius": 10.0,
        "scale": 1e6,
        "background": 10.0,
    }

    y_true = sphere_form_factor(
        q_data,
        true_params["radius"],
        true_params["scale"],
        true_params["background"],
    )
    noise = 0.05 * np.sqrt(np.maximum(y_true, 1)) * np.random.randn(len(q_data))
    y_data = np.maximum(y_true + noise, 0.1)

    print(f"  Data points: {len(q_data)}")
    print(f"  True radius: {true_params['radius']:.2f} nm")
    print(f"  True scale:  {true_params['scale']:.2e}")
    print()

    p0 = [8.0, 5e5, 5.0]
    bounds = (
        [1.0, 1e3, 0.0],
        [50.0, 1e9, 100.0],
    )

    print("Fitting sphere form factor with SAXS preset...")
    popt, pcov = fit(
        sphere_form_factor,
        q_data,
        y_data,
        p0=p0,
        bounds=bounds,
        **kwargs,
    )

    print()
    print("Fit Results:")
    print("-" * 40)
    print(f"  radius:     {popt[0]:.4f} nm (true: {true_params['radius']:.4f})")
    print(f"  scale:      {popt[1]:.4e} (true: {true_params['scale']:.4e})")
    print(f"  background: {popt[2]:.4f} (true: {true_params['background']:.4f})")

    if pcov is not None:
        perr = np.sqrt(np.diag(pcov))
        print()
        print("Parameter Uncertainties (1-sigma):")
        print(f"  radius:     +/- {perr[0]:.4f} nm")
        print(f"  scale:      +/- {perr[1]:.4e}")
        print(f"  background: +/- {perr[2]:.4f}")

    rel_error = 100 * np.abs(popt[0] - true_params["radius"]) / true_params["radius"]
    print()
    print(f"  Radius relative error: {rel_error:.3f}%")

    print()
    print("=" * 70)
    print("Notes on SAXS Preset Customization")
    print("=" * 70)
    print()
    print("The SAXS preset uses the kwargs factory pattern with adjustments for:")
    print()
    print("1. Tighter tolerances (1e-9)")
    print("   - Form factor oscillations provide good gradient information")
    print("   - Accurate size determination requires higher precision")
    print("   - Still computationally efficient for typical SAXS datasets")
    print()
    print("2. Multi-start optimization (n_starts=12)")
    print("   - Helps with polydisperse systems")
    print("   - Useful for complex form factors (core-shell, ellipsoids)")
    print("   - Moderate number sufficient for well-conditioned problems")
    print()
    print("3. Parameter scaling considerations:")
    print("   - Intensity scales span many orders of magnitude")
    print("   - NLSQ's automatic normalization handles this well")
    print("   - Use bounds to constrain physically reasonable values")
    print()
    print("4. Common SAXS models:")
    print("   - Sphere, cylinder, ellipsoid form factors")
    print("   - Core-shell and multi-shell structures")
    print("   - Structure factors for concentrated systems")
    print()

In [ ]:
if __name__ == "__main__":
    main()